In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
movies = pd.read_csv('C:\Acmegrade\movies.csv')
credits = pd.read_csv('C:\Acmegrade\credits.csv')
movies = movies.merge(credits, on='title')


In [4]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.dropna(inplace=True)


In [5]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        return str.lower(x.replace(" ", ""))
features = ['genres', 'keywords', 'cast', 'crew', 'overview']
for feature in features:
    movies[feature] = movies[feature].apply(clean_data)
movies['combined_features'] = movies.apply(lambda x: ' '.join(x[feature] for feature in features), axis=1)


In [6]:

cv = CountVectorizer(max_features=5000, stop_words='english')
count_matrix = cv.fit_transform(movies['combined_features'])
cosine_sim = cosine_similarity(count_matrix)


In [14]:

def get_recommendations(title, cosine_sim=cosine_sim):
    
    title = title.lower()
    if title in movies['title'].str.lower().values:
        idx = movies[movies['title'].str.lower() == title].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]
        return movies['title'].iloc[movie_indices]
    else:
        return "Movie not found in the dataset."
print(get_recommendations('Batman'))


4271                                    Batman
3738                               Night Watch
1752    Star Trek VI: The Undiscovered Country
2144                                     Risen
2925                              Major League
859                               Thunderbirds
627                            The Last Legion
1551                         The Addams Family
1361                                    Batman
2048                                 RoboCop 3
Name: title, dtype: object
